In [1]:
import numpy as np
from skyfield.api import load, wgs84, Distance
from skyfield.toposlib import ITRSPosition
from skyfield.framelib import itrs
from los_intercept import *


import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
tles = gen_sats(sat_nos=[48915])
sat = tles[0][0]
times = gen_times(start_yr=2021, start_mo=11, start_day=27, days=1, step_min=1)


Satellite(s) Loaded from TLE:
[<EarthSatellite YAM-3 catalog #48915 epoch 2021-11-27 14:32:26 UTC>]
Propogation time: 
 2021-11-27 00:00:00+00:00 
to 
 2021-11-27 23:59:00+00:00


In [3]:
# %%timeit
los_lat, los_lon, d = get_los(sat, times[0])

In [4]:
geo = sat.at(times[0])
geos = sat.at(times)

lat, lon = wgs84.latlon_of(geo)

In [5]:
print(lat)
print(los_lat)

print(lon)
print(los_lon)

print(d)

-57deg 05' 02.0"
-57deg 05' 52.6"
43deg 12' 59.2"
43deg 12' 59.2"
555.3654337020286


In [6]:
# %%timeit
los_lat, los_lon, d, lvlh, pointing = get_los2(sat, times[0])

In [7]:
# https://github.com/skyfielders/python-skyfield/issues/492
# https://github.com/skyfielders/python-skyfield/issues/557

lvlh

{'X': array([0.77441492, 0.3517825 , 0.52586178]),
 'Y': array([-0.49129692,  0.85806636,  0.14949733]),
 'Z': array([-0.39774739, -0.37372447,  0.83793021])}

In [8]:
np.cross(lvlh["X"], lvlh["Y"])

array([-0.39863376, -0.37412724,  0.83732905])

In [9]:
np.degrees(np.arccos(np.dot(lvlh["X"], lvlh["Z"])))

89.93444077095597

In [10]:
np.degrees(np.arccos(np.dot(lvlh["X"], lvlh["Y"])))


90.0

In [11]:
from scipy.spatial.transform import Rotation as R

vec = pointing

rotation_degrees = 10
rotation_radians = np.radians(rotation_degrees)
rotation_axis = np.array(lvlh["X"])

rotation_vector = rotation_radians * rotation_axis
rotation = R.from_rotvec(rotation_vector)
rotated_vec = rotation.apply(vec)


print(rotated_vec)

[-0.30637849 -0.51704221  0.79924938]


In [12]:
np.linalg.norm(rotated_vec)

1.0

In [13]:
hfov_x = 10
hfov_y = 15
corners_xyz = {}

for c, r, a in zip(["c1", "c2", "c3", "c4"], [hfov_x, -hfov_x, hfov_y, -hfov_y], [lvlh["X"],  lvlh["X"], lvlh["Y"], lvlh["Y"]]):
    vec = pointing

    rotation_degrees = r
    rotation_radians = np.radians(rotation_degrees)
    rotation_axis = np.array(a)

    rotation_vector = rotation_radians * rotation_axis
    rotation = R.from_rotvec(rotation_vector)
    corners_xyz[c] = rotation.apply(vec)

In [14]:
corners_xyz

{'c1': array([-0.30637849, -0.51704221,  0.79924938]),
 'c2': array([-0.47700401, -0.21903908,  0.85116923]),
 'c3': array([-0.18364323, -0.26983137,  0.94523341]),
 'c4': array([-0.58474573, -0.45214887,  0.67352345])}

In [15]:
polys = []
for n in range(len(times)):
    time = times[n]
    # print("Acquisition time: {}".format(time.utc_datetime()))
    los_lat, los_lon, d, lvlh, pointing = get_los2(sat, time)

    corners_lla = {}

    for c, r, a in zip(["c1", "c2", "c3", "c4"], [hfov_x, hfov_y, -hfov_x, -hfov_y], [lvlh["X"],  lvlh["Y"], lvlh["X"], lvlh["Y"]]):
        vec = pointing

        rotation_degrees = r
        rotation_radians = np.radians(rotation_degrees)
        rotation_axis = a

        rotation_vector = rotation_radians * rotation_axis
        rotation = R.from_rotvec(rotation_vector)
        # corners_xyz[c] = rotation.apply(vec) # !!
        corners_xyz = rotation.apply(vec)

        # for c_read in corners_xyz:
        xyz_rates = sat.at(time).frame_xyz_and_velocity(itrs)
        xyz_dist = xyz_rates[0]
        # los_xyz = los_to_earth(xyz_dist.km, corners_xyz[c]) # !!
        los_xyz = los_to_earth(xyz_dist.km, corners_xyz)

        los = Distance(km=los_xyz)
        los_itrs = ITRSPosition(los)
        los_itrs.at(time).frame_xyz(itrs).km

        los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
        # los = Distance(km=corners_xyz[c]) # !!
        # los = Distance(km=corners_xyz)
        # los_itrs = ITRSPosition(los)
        # los_itrs.at(time).frame_xyz(itrs).km

        los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
        # print(los_lat, los_lon)
        # d = np.sqrt(np.sum(np.square(xyz_dist.km - los_xyz)))
        corners_lla[c, "lat"] = los_lat.degrees
        corners_lla[c, "lon"] = los_lon.degrees

    polys.append(Polygon([(corners_lla["c1", "lon"], corners_lla["c1", "lat"]), 
            (corners_lla["c2", "lon"], corners_lla["c2", "lat"]), 
            (corners_lla["c3", "lon"], corners_lla["c3", "lat"]),
            (corners_lla["c4", "lon"], corners_lla["c4", "lat"]),
            (corners_lla["c1", "lon"], corners_lla["c1", "lat"])]
            # (gdf.loc[n].bound1_lon, gdf.loc[n].bound1_lat)
            ))

In [16]:
print(pointing)
corners_lla

[-0.78528743 -0.53457134  0.31234137]


{('c1', 'lat'): -18.482121004139014,
 ('c1', 'lon'): 33.3691612190201,
 ('c2', 'lat'): -17.044059877940324,
 ('c2', 'lon'): 33.97652437916407,
 ('c3', 'lat'): -18.143399010074525,
 ('c3', 'lon'): 35.11779996456349,
 ('c4', 'lat'): -19.586567845087476,
 ('c4', 'lon'): 34.51655166490876}

In [17]:
poly_df = gpd.GeoDataFrame(
        data=polys, 
        columns=['geometry'], 
        crs="EPSG:4326"
        )

In [18]:
poly_df.explore()

In [19]:
cs_dict = {"c1": {"X" : -hfov_x, "Y": hfov_y}, 
    "c2": {"X" : hfov_x, "Y": hfov_y},
    "c3": {"X" : hfov_x, "Y": -hfov_y},
    "c4": {"X" : -hfov_x, "Y": -hfov_y}
}

cs_lla_dict = {"c1": {"lat" : None, "lon": None}, 
    "c2": {"lat" : None, "lon": None},
    "c3": {"lat" : None, "lon": None},
    "c4": {"lat" : None, "lon": None}
}

cs_dict["c1"]["X"]

-10

In [20]:
polys = []

for time in times:

    SSP_lat, SSP_lon, SSP_d, lvlh, vec = get_los2(sat, time)
    xyz_rates = sat.at(time).frame_xyz_and_velocity(itrs)
    xyz_dist = xyz_rates[0]

    for c in cs_dict:
        # for axis in ["X", "Y"]:
        rot_X_deg = cs_dict[c]["X"]
        rot_X_rad = np.radians(rot_X_deg)
        rot_X_ax = lvlh["X"]

        rot_Y_deg = cs_dict[c]["Y"]
        rot_Y_rad = np.radians(rot_Y_deg)
        rot_Y_ax = lvlh["Y"]

        # Rotate about X
        rot_X_vec = rot_X_rad * rot_X_ax
        rot_X = R.from_rotvec(rot_X_vec)
        los_X = rot_X.apply(vec)

        # Rotate about Y
        rot_Y_vec = rot_Y_rad * rot_Y_ax
        rot_Y = R.from_rotvec(rot_Y_vec)
        los_XY = rot_Y.apply(los_X)

        los_xyz = los_to_earth(xyz_dist.km, los_XY)
        los = Distance(km=los_xyz)
        los_itrs = ITRSPosition(los)
        los_itrs.at(time).frame_xyz(itrs).km

        los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
        cs_lla_dict[c]["lat"] = los_lat.degrees
        cs_lla_dict[c]["lon"] = los_lon.degrees

    polys.append(Polygon([(cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"]), 
            (cs_lla_dict["c2"]["lon"], cs_lla_dict["c2"]["lat"]), 
            (cs_lla_dict["c3"]["lon"], cs_lla_dict["c3"]["lat"]),
            (cs_lla_dict["c4"]["lon"], cs_lla_dict["c4"]["lat"]),
            (cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"])]
            # (gdf.loc[n].bound1_lon, gdf.loc[n].bound1_lat)
            ))
        

In [21]:
cs_lla_dict

{'c1': {'lat': -16.865478960597628, 'lon': 34.876642078948315},
 'c2': {'lat': -17.214477654008785, 'lon': 33.07372954570755},
 'c3': {'lat': -19.763453189137106, 'lon': 33.60076673910523},
 'c4': {'lat': -19.408664483642696, 'lon': 35.43095984195202}}

In [22]:
poly_df = gpd.GeoDataFrame(
        data=polys, 
        columns=['geometry'], 
        crs="EPSG:4326"
        )

In [26]:
poly_plot = poly_df.geometry.to_crs("ESRI:54009")
poly_plot = poly_plot[poly_plot.area < poly_plot.area.quantile(.98)]
poly_plot.explore()